# Load CSV Fundamental Data into Zipline Custom Database

This notebook demonstrates how to:
1. Load fundamental data from CSV files
2. Map symbols to Zipline SIDs
3. Create a custom SQLite database
4. Use the data in Zipline Pipeline

This is a zipline-reloaded native approach (no QuantRocket dependencies).

## 1. Setup and Imports

In [ ]:
import os
import glob
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path

# Zipline imports
from zipline.data.bundles import load as load_bundle, register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle
from zipline.pipeline import Pipeline
from zipline.pipeline.data.db import Database, Column

# Register Sharadar bundle (in case extension.py didn't load)
try:
    # Try to register the bundle
    register(
        'sharadar',
        sharadar_bundle(
            tickers=None,
            incremental=True,
            include_funds=True,
        ),
    )
    print("✓ Registered Sharadar bundle")
except Exception as e:
    # Bundle may already be registered
    print(f"✓ Sharadar bundle already registered (or error: {e})")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 120)

print("✓ Imports complete")

## 2. Configuration

Set your database name and data directory paths.

In [ ]:
# Configuration
DATABASE_NAME = "refe-fundamentals"  # Name for your custom database
DATA_DIR = "/data/csv/"  # Directory with CSV files (persistent across Docker restarts)
VIX_SIGNAL_PATH = "/data/csv/vix_flag.csv"  # Optional VIX signal data

# Database will be created in ~/.zipline/data/custom/
DB_DIR = Path.home() / '.zipline' / 'data' / 'custom'
DB_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = DB_DIR / f"{DATABASE_NAME}.sqlite"

print(f"Database will be created at: {DB_PATH}")
print(f"Looking for CSV files in: {DATA_DIR}")
print(f"\n💡 Tip: Place your CSV files in /data/csv/ (inside container)")
print(f"   or ./data/csv/ (on host machine) for persistent storage")

## 3. Define Database Schema

Define the columns that will be in your custom database.

In [ ]:
# Define your database schema
# This matches the columns from the QuantRocket example
SCHEMA = {
    'Symbol': 'TEXT',
    'Sid': 'INTEGER',
    'Date': 'TEXT',
    'RefPriceClose': 'REAL',
    'RefVolume': 'REAL',
    'CompanyCommonName': 'TEXT',
    'EnterpriseValue_DailyTimeSeries_': 'REAL',
    'CompanyMarketCap': 'REAL',
    'GICSSectorName': 'TEXT',
    'FOCFExDividends_Discrete': 'REAL',
    'InterestExpense_NetofCapitalizedInterest': 'REAL',
    'Debt_Total': 'REAL',
    'EarningsPerShare_Actual': 'REAL',
    'EarningsPerShare_SmartEstimate_prev_Q': 'REAL',
    'EarningsPerShare_ActualSurprise': 'REAL',
    'EarningsPerShare_SmartEstimate_current_Q': 'REAL',
    'LongTermGrowth_Mean': 'REAL',
    'PriceTarget_Median': 'REAL',
    'CombinedAlphaModelSectorRank': 'REAL',
    'CombinedAlphaModelSectorRankChange': 'REAL',
    'CombinedAlphaModelRegionRank': 'REAL',
    'TradeDate': 'TEXT',
    'EPS_SurpirsePrct_prev_Q': 'REAL',
    'Estpricegrowth_percent': 'REAL',
    'CashFlowComponent_Current': 'REAL',
    'EarningsQualityRegionRank_Current': 'REAL',
    'EnterpriseValueToEBIT_DailyTimeSeriesRatio_': 'REAL',
    'EnterpriseValueToEBITDA_DailyTimeSeriesRatio_': 'REAL',
    'EnterpriseValueToSales_DailyTimeSeriesRatio_': 'REAL',
    'Dividend_Per_Share_SmartEstimate': 'REAL',
    'CashFlowPerShare_BrokerEstimate': 'REAL',
    'FreeCashFlowPerShare_BrokerEstimate': 'REAL',
    'ForwardPEG_DailyTimeSeriesRatio_': 'REAL',
    'PriceEarningsToGrowthRatio_SmartEstimate_': 'REAL',
    'ReturnOnInvestedCapital_BrokerEstimate': 'REAL',
    'Recommendation_NumberOfTotal': 'REAL',
    'Recommendation_Median_1_5_': 'REAL',
    'Recommendation_NumberOfStrongBuy': 'REAL',
    'Recommendation_NumberOfBuy': 'REAL',
    'Recommendation_Mean_1_5_': 'REAL',
    'ReturnOnCapitalEmployed_Actual': 'REAL',
    'GrossProfitMargin_': 'REAL',
    'ReturnOnEquity_SmartEstimat': 'REAL',
    'ReturnOnAssets_SmartEstimate': 'REAL',
    'CashCashEquivalents_Total': 'REAL',
    'ForwardPriceToCashFlowPerShare_DailyTimeSeriesRatio_': 'REAL',
    'ForwardPriceToSalesPerShare_DailyTimeSeriesRatio_': 'REAL',
    'ForwardEnterpriseValueToOperatingCashFlow_DailyTimeSeriesRatio_': 'REAL',
    'GrossProfitMargin_ActualSurprise': 'REAL',
    'pred': 'REAL',  # VIX signal
}

print(f"✓ Schema defined with {len(SCHEMA)} columns")

## 4. Load CSV Files

Load all CSV files from the data directory and concatenate them.

In [ ]:
# Find all CSV files
os.chdir(DATA_DIR)
csv_files = sorted(glob.glob('*.csv'))

print(f"Found {len(csv_files)} CSV files:")
for f in csv_files[:5]:  # Show first 5
    print(f"  - {f}")
if len(csv_files) > 5:
    print(f"  ... and {len(csv_files) - 5} more")

# Load and concatenate all CSV files
print("\nLoading CSV files...")
custom_data = pd.DataFrame()

for csv_file in csv_files:
    print(f"  Loading {csv_file}...")
    df = pd.read_csv(os.path.join(DATA_DIR, csv_file))
    custom_data = pd.concat([custom_data, df], ignore_index=True)

print(f"\n✓ Loaded {len(custom_data):,} total rows")
print(f"Date range: {custom_data['Date'].min()} to {custom_data['Date'].max()}")
print(f"Unique symbols: {custom_data['Symbol'].nunique()}")

# Show sample
print("\nSample data:")
custom_data.head()

## 5. Optional: Load Recent Data Only

To reduce memory usage, you can filter to recent data only.

In [ ]:
# Optional: Keep only recent data (e.g., last 600,000 rows)
# Comment out if you want all historical data
RECENT_ROWS = 600000

if len(custom_data) > RECENT_ROWS:
    print(f"Filtering to most recent {RECENT_ROWS:,} rows...")
    custom_data = custom_data.tail(RECENT_ROWS).copy()
    print(f"✓ Filtered. New date range: {custom_data['Date'].min()} to {custom_data['Date'].max()}")
else:
    print(f"Dataset has {len(custom_data):,} rows - no filtering needed")

## 6. Map Symbols to Zipline SIDs

Map your symbols to Zipline Security IDs (SIDs) using the asset finder.

In [ ]:
# Load the Sharadar bundle to get the asset finder
print("Loading Sharadar bundle...")

# Load bundle with current timestamp
# This gives us access to the asset database
bundle_timestamp = pd.Timestamp.now(tz='UTC')
bundle_data = load_bundle('sharadar', timestamp=bundle_timestamp)
asset_finder = bundle_data.asset_finder

# Get all equities
print("Mapping symbols to SIDs...")
all_assets = asset_finder.retrieve_all(asset_finder.sids)

# Create symbol -> sid mapping
symbol_to_sid = {}
for asset in all_assets:
    if hasattr(asset, 'symbol'):
        symbol_to_sid[asset.symbol] = asset.sid

print(f"✓ Found {len(symbol_to_sid):,} symbols in bundle")

# Map SIDs to your data
custom_data['Sid'] = custom_data['Symbol'].map(symbol_to_sid)

# Check mapping success
mapped = custom_data['Sid'].notna().sum()
unmapped = custom_data['Sid'].isna().sum()

print(f"\nMapping results:")
print(f"  Mapped: {mapped:,} rows ({mapped/len(custom_data)*100:.1f}%)")
print(f"  Unmapped: {unmapped:,} rows ({unmapped/len(custom_data)*100:.1f}%)")

if unmapped > 0:
    unmapped_symbols = custom_data[custom_data['Sid'].isna()]['Symbol'].unique()
    print(f"\n  Unmapped symbols (first 10): {list(unmapped_symbols[:10])}")
    print(f"  Tip: These symbols may not be in the Sharadar bundle")

# Remove unmapped rows
custom_data = custom_data[custom_data['Sid'].notna()].copy()
print(f"\n✓ Kept {len(custom_data):,} mapped rows")

## 7. Merge VIX Signal Data (Optional)

If you have additional data like VIX signals, merge it here.

In [ ]:
# Load VIX signal data if available
if os.path.exists(VIX_SIGNAL_PATH):
    print(f"Loading VIX signal from {VIX_SIGNAL_PATH}...")
    vix_signal = pd.read_csv(VIX_SIGNAL_PATH)
    
    # Standardize column names
    vix_signal.rename(columns={'symbol': 'Symbol', 'date': 'Date'}, inplace=True)
    vix_signal['Date'] = pd.to_datetime(vix_signal['Date'])
    
    # Merge with custom data
    custom_data['Date'] = pd.to_datetime(custom_data['Date'])
    custom_data = pd.merge(custom_data, vix_signal[['Symbol', 'Date', 'pred']], 
                          on=['Symbol', 'Date'], how='left')
    
    print(f"✓ Merged VIX signal data")
else:
    print(f"VIX signal file not found at {VIX_SIGNAL_PATH}")
    print("Skipping VIX merge (this is optional)")

## 8. Data Cleaning

Clean and prepare data for database insertion.

In [ ]:
print("Cleaning data...")

# Ensure Date is datetime
custom_data['Date'] = pd.to_datetime(custom_data['Date'])

# Forward fill missing values by symbol
print("  Forward filling missing values by symbol...")
for col in custom_data.columns:
    if col not in ['Symbol', 'Sid', 'Date']:
        custom_data[col] = custom_data.groupby('Symbol')[col].transform(lambda x: x.ffill())

# Handle sector - fill empty strings instead of NaN
custom_data['GICSSectorName'] = custom_data['GICSSectorName'].fillna('')

# Fill remaining NaNs with 0 (for numeric columns)
print("  Filling remaining NaN values...")
custom_data = custom_data.fillna(0)

# Convert Sid to integer
custom_data['Sid'] = custom_data['Sid'].astype(int)

# Sort by date and symbol
custom_data = custom_data.sort_values(['Date', 'Symbol'])

print(f"✓ Data cleaned")
print(f"\nFinal dataset:")
print(f"  Rows: {len(custom_data):,}")
print(f"  Columns: {len(custom_data.columns)}")
print(f"  Date range: {custom_data['Date'].min()} to {custom_data['Date'].max()}")
print(f"  Symbols: {custom_data['Symbol'].nunique()}")

# Show sample
print("\nSample cleaned data:")
custom_data.head()

## 9. Create SQLite Database

Create the custom SQLite database in Zipline format.

In [ ]:
print(f"Creating database at {DB_PATH}...")

# Remove existing database if it exists
if DB_PATH.exists():
    print(f"  Removing existing database...")
    DB_PATH.unlink()

# Create database connection
conn = sqlite3.connect(str(DB_PATH))
cursor = conn.cursor()

# Create table with schema
columns_def = ', '.join([f'"{col}" {dtype}' for col, dtype in SCHEMA.items()])
create_table_sql = f'''
CREATE TABLE fundamentals (
    {columns_def}
);
'''

print("  Creating table...")
cursor.execute(create_table_sql)

# Create indices for fast lookups
print("  Creating indices...")
cursor.execute('CREATE INDEX idx_sid ON fundamentals(Sid);')
cursor.execute('CREATE INDEX idx_date ON fundamentals(Date);')
cursor.execute('CREATE INDEX idx_sid_date ON fundamentals(Sid, Date);')

# Insert data
print(f"  Inserting {len(custom_data):,} rows...")

# Prepare data for insertion - only use columns that exist in custom_data
# Add missing columns with default values (0 for numeric, empty string for text)
insert_data = custom_data.copy()

# Add any missing schema columns with appropriate defaults
for col, dtype in SCHEMA.items():
    if col not in insert_data.columns:
        if dtype == 'TEXT':
            insert_data[col] = ''
        else:  # REAL or INTEGER
            insert_data[col] = 0
        print(f"  Added missing column '{col}' with default values")

# Select only the columns in the schema (in the correct order)
insert_data = insert_data[list(SCHEMA.keys())].copy()

# Convert Date to string format for SQLite
insert_data['Date'] = insert_data['Date'].dt.strftime('%Y-%m-%d')

# Insert in chunks for better performance
chunk_size = 10000
total_chunks = (len(insert_data) + chunk_size - 1) // chunk_size

for i in range(0, len(insert_data), chunk_size):
    chunk = insert_data.iloc[i:i+chunk_size]
    chunk.to_sql('fundamentals', conn, if_exists='append', index=False)
    
    chunk_num = i // chunk_size + 1
    if chunk_num % 10 == 0 or chunk_num == total_chunks:
        print(f"    Inserted chunk {chunk_num}/{total_chunks} ({i+len(chunk):,} rows)...")

conn.commit()
conn.close()

print(f"\n✓ Database created successfully!")
print(f"  Path: {DB_PATH}")
print(f"  Size: {DB_PATH.stat().st_size / 1024 / 1024:.1f} MB")

## 10. Define Database Class

Create a Database class to use this data in Zipline Pipeline.

In [ ]:
# Define the Database class
class REFEFundamentals(Database):
    """
    Custom REFE Fundamentals database.
    
    Usage in Pipeline:
        roe = REFEFundamentals.ReturnOnEquity_SmartEstimat.latest
        sector = REFEFundamentals.GICSSectorName.latest
    """
    
    CODE = DATABASE_NAME
    LOOKBACK_WINDOW = 252  # Days to look back
    
    # Price and volume
    RefPriceClose = Column(float)
    RefVolume = Column(float)
    
    # Company info
    CompanyCommonName = Column(str)
    GICSSectorName = Column(str)
    
    # Valuation metrics
    EnterpriseValue_DailyTimeSeries_ = Column(float)
    CompanyMarketCap = Column(float)
    
    # Cash flow
    FOCFExDividends_Discrete = Column(float)
    CashFlowComponent_Current = Column(float)
    CashFlowPerShare_BrokerEstimate = Column(float)
    FreeCashFlowPerShare_BrokerEstimate = Column(float)
    
    # Debt and interest
    InterestExpense_NetofCapitalizedInterest = Column(float)
    Debt_Total = Column(float)
    
    # Earnings
    EarningsPerShare_Actual = Column(float)
    EarningsPerShare_SmartEstimate_prev_Q = Column(float)
    EarningsPerShare_ActualSurprise = Column(float)
    EarningsPerShare_SmartEstimate_current_Q = Column(float)
    EPS_SurpirsePrct_prev_Q = Column(float)
    
    # Growth and targets
    LongTermGrowth_Mean = Column(float)
    PriceTarget_Median = Column(float)
    Estpricegrowth_percent = Column(float)
    
    # Rankings
    CombinedAlphaModelSectorRank = Column(float)
    CombinedAlphaModelSectorRankChange = Column(float)
    CombinedAlphaModelRegionRank = Column(float)
    EarningsQualityRegionRank_Current = Column(float)
    
    # Ratios
    EnterpriseValueToEBIT_DailyTimeSeriesRatio_ = Column(float)
    EnterpriseValueToEBITDA_DailyTimeSeriesRatio_ = Column(float)
    EnterpriseValueToSales_DailyTimeSeriesRatio_ = Column(float)
    ForwardPEG_DailyTimeSeriesRatio_ = Column(float)
    PriceEarningsToGrowthRatio_SmartEstimate_ = Column(float)
    ForwardPriceToCashFlowPerShare_DailyTimeSeriesRatio_ = Column(float)
    ForwardPriceToSalesPerShare_DailyTimeSeriesRatio_ = Column(float)
    ForwardEnterpriseValueToOperatingCashFlow_DailyTimeSeriesRatio_ = Column(float)
    
    # Returns
    ReturnOnInvestedCapital_BrokerEstimate = Column(float)
    ReturnOnCapitalEmployed_Actual = Column(float)
    ReturnOnEquity_SmartEstimat = Column(float)
    ReturnOnAssets_SmartEstimate = Column(float)
    
    # Margins
    GrossProfitMargin_ = Column(float)
    GrossProfitMargin_ActualSurprise = Column(float)
    
    # Analyst recommendations
    Recommendation_NumberOfTotal = Column(float)
    Recommendation_Median_1_5_ = Column(float)
    Recommendation_NumberOfStrongBuy = Column(float)
    Recommendation_NumberOfBuy = Column(float)
    Recommendation_Mean_1_5_ = Column(float)
    
    # Cash
    CashCashEquivalents_Total = Column(float)
    
    # Dividends
    Dividend_Per_Share_SmartEstimate = Column(float)
    
    # VIX prediction signal
    pred = Column(float)


print("✓ REFEFundamentals Database class defined")
print(f"  Database code: {REFEFundamentals.CODE}")
print(f"  Lookback window: {REFEFundamentals.LOOKBACK_WINDOW} days")
print(f"  Columns defined: {len([attr for attr in dir(REFEFundamentals) if isinstance(getattr(REFEFundamentals, attr), Column)])}")

print("\nExample usage:")
print("  roe = REFEFundamentals.ReturnOnEquity_SmartEstimat.latest")
print("  pe_growth = REFEFundamentals.PriceEarningsToGrowthRatio_SmartEstimate_.latest")
print("  sector = REFEFundamentals.GICSSectorName.latest")

## 11. Verify Database

Query the database to verify data was loaded correctly.

In [ ]:
# Connect and query
conn = sqlite3.connect(str(DB_PATH))

# Get row count
row_count = pd.read_sql("SELECT COUNT(*) as count FROM fundamentals", conn).iloc[0, 0]
print(f"Total rows in database: {row_count:,}")

# Get date range
date_range = pd.read_sql("SELECT MIN(Date) as min_date, MAX(Date) as max_date FROM fundamentals", conn)
print(f"Date range: {date_range.iloc[0, 0]} to {date_range.iloc[0, 1]}")

# Get symbol count
symbol_count = pd.read_sql("SELECT COUNT(DISTINCT Symbol) as count FROM fundamentals", conn).iloc[0, 0]
print(f"Unique symbols: {symbol_count:,}")

# Show sample data for a specific symbol
print("\nSample data for AAPL:")
aapl_data = pd.read_sql("""
    SELECT Date, Symbol, RefPriceClose, CompanyMarketCap, 
           ReturnOnEquity_SmartEstimat, PriceTarget_Median
    FROM fundamentals 
    WHERE Symbol = 'AAPL' 
    ORDER BY Date DESC 
    LIMIT 5
""", conn)
print(aapl_data)

print("\nSample data for IBM:")
ibm_data = pd.read_sql("""
    SELECT Date, Symbol, RefPriceClose, CompanyMarketCap, 
           ReturnOnEquity_SmartEstimat, GICSSectorName
    FROM fundamentals 
    WHERE Symbol = 'IBM' 
    ORDER BY Date DESC 
    LIMIT 5
""", conn)
print(ibm_data)

conn.close()

print("\n✓ Database verification complete")

## 12. Usage Example

Example of how to use this database in a backtest.

In [ ]:
print("To use this database in your backtests:")
print("\n1. Import the Database class:")
print("   from zipline.pipeline.data.db import Database, Column")
print("\n2. Define the REFEFundamentals class (from cell 10 above)")
print("\n3. Use in your pipeline:")
print("   ")
print("   def make_pipeline():")
print("       roe = REFEFundamentals.ReturnOnEquity_SmartEstimat.latest")
print("       growth = REFEFundamentals.LongTermGrowth_Mean.latest")
print("       sector = REFEFundamentals.GICSSectorName.latest")
print("       ")
print("       # Screen for quality companies")
print("       quality = (roe > 15) & (growth > 10)")
print("       ")
print("       return Pipeline(")
print("           columns={")
print("               'ROE': roe,")
print("               'Growth': growth,")
print("               'Sector': sector,")
print("           },")
print("           screen=quality")
print("       )")
print("\n4. The CustomSQLiteLoader will automatically load data based on REFEFundamentals.CODE")

print("\n✓ Setup complete! Your custom fundamentals database is ready to use.")

## Summary

This notebook:
1. ✅ Loaded CSV files with fundamental data
2. ✅ Mapped symbols to Zipline SIDs using the asset finder
3. ✅ Cleaned and prepared the data
4. ✅ Created a custom SQLite database in ~/.zipline/data/custom/
5. ✅ Defined a Database class for use in Pipeline
6. ✅ Verified the database contents

The database is now ready to use in your Zipline backtests with the CustomSQLiteLoader.

**Next steps:**
- Copy the REFEFundamentals class definition to your backtest algorithm
- Use REFEFundamentals.ColumnName.latest in your pipeline
- The backtest_helpers.py will automatically detect and load the data